<a href="https://colab.research.google.com/github/pxyzP/phishoff/blob/main/phishoffbert_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Phish-Off bert model**

**1. Import package ต่างๆ ที่จำเป็น**

In [ ]:
import pandas as pd
import itertools
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMClassifier
import os
import seaborn as sns
from wordcloud import WordCloud



Mounted at /content/gdrive
(651191, 2)


**2. download dataset ทั้ง 2 จาก google drive**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
urlsdata1 = pd.read_csv('/content/gdrive/My Drive/malurl/malicious_phish.csv')
urlsdata1.head()
print(urlsdata1.shape)
import pandas as pd
urlsdata2 = pd.read_csv('/content/gdrive/My Drive/malurl/urldatagoodbad.csv')
urlsdata2.head()

,url,type
0,diaryofagameaddict.com,bad
1,diaryofagameaddict.com,bad
2,diaryofagameaddict.com,bad
3,diaryofagameaddict.com,bad
4,diaryofagameaddict.com,bad


In [ ]:
urlsdata2.type.value_counts()

good    344821
bad      75643
Name: type, dtype: int64

**define dictionary ขึ้นเพื่อแปลง
benign, good --> safe
phishing, malware, defacement, bad --> unsafe**

In [ ]:
#Define a dictionary
mapping = {
    'benign': 'safe',
    'defacement': 'unsafe',
    'phishing': 'unsafe',
    'malware' : 'unsafe'
}

urlsdata1 = urlsdata1.replace({'type': mapping})

#Save the edited data
urlsdata1.to_csv('new_urlsdataset1.csv', index=False)

In [ ]:
#Define a dictionary
mapping2 = {
    'good': 'safe',
    'bad': 'unsafe'
}

urlsdata2 = urlsdata2.replace({'type': mapping2})

#Save the edited data
urlsdata2.to_csv('new_urlsdataset2.csv', index=False)

In [ ]:
urlsdata1 = pd.read_csv('new_urlsdataset1.csv')
urlsdata2 = pd.read_csv('new_urlsdataset2.csv')

finaldata = pd.concat([urlsdata1, urlsdata2])

#rename the 1st column
finaldata = finaldata.rename(columns={'type_x': 'safeness'})
finaldata.to_csv('finaldata.csv', index=False)


# 4. extract ข้อมูลดังนี้


IP ADRESS, ABNORMAL URL, COUNTING DOTS, COUNTING WWW, COUNT AT, COUNT SLASH, COUNT DOUBLE SLASH, CHECK FOR SUSPICIOUS WORDS, URLS ABBRIEVATION, COUNT HTTPS, COUNT HTTP, COUNT PERCENT, COUNT QUESTION MARK, COUNT DASH, COUNT EQUAL, COUNT DIGITS, COUNT LETTERS, URL LENGTH, HOSTNAME LENGTH, FIRST DIRECTORY LENGTH, LENGTH OF THE TOP-LEVEL DOMAIN

In [ ]:
import re
#Use of IP or not in domain
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        return 0
finaldata['ipaddress'] = finaldata['url'].apply(lambda i: having_ip_address(i))
from urllib.parse import urlparse
def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        # print match.group()
        return 1
    else:
        return 0
finaldata['sus_url'] = finaldata['url'].apply(lambda i: abnormal_url(i))
def count_dot(url):
    count_dot = url.count('.')
    return count_dot
finaldata['dotnum'] = finaldata['url'].apply(lambda i: count_dot(i))
finaldata.head()


,url,type,ipaddress,sus_url,dotnum
0,br-icloud.com.br,unsafe,0,0,2
1,mp3raid.com/music/krizz_kaliko.html,safe,0,0,2
2,bopsecrets.org/rexroth/cr/1.htm,safe,0,0,2
3,http://www.garage-pirenne.be/index.php?option=...,unsafe,0,1,3
4,http://adventure-nicaragua.net/index.php?optio...,unsafe,0,1,2


In [ ]:
def count_www(url):
    url.count('www')
    return url.count('www')
finaldata['wwwnum'] = finaldata['url'].apply(lambda i: count_www(i))
def count_atrate(url):

    return url.count('@')
finaldata['atnum'] = finaldata['url'].apply(lambda i: count_atrate(i))
def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')
finaldata['doubleslashnum'] = finaldata['url'].apply(lambda i: no_of_dir(i))
def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')
finaldata['embeddomainnnum'] = finaldata['url'].apply(lambda i: no_of_embed(i))
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0

finaldata['abbreivation'] = finaldata['url'].apply(lambda i: shortening_service(i))
def count_https(url):
    return url.count('https')
finaldata['count-https'] = finaldata['url'].apply(lambda i : count_https(i))
def count_http(url):
    return url.count('http')
finaldata['count-http'] = finaldata['url'].apply(lambda i : count_http(i))
def count_per(url):
    return url.count('%')
finaldata['count%'] = finaldata['url'].apply(lambda i : count_per(i))
def count_ques(url):
    return url.count('?')
finaldata['count?'] = finaldata['url'].apply(lambda i: count_ques(i))
def count_hyphen(url):
    return url.count('-')
finaldata['count-'] = finaldata['url'].apply(lambda i: count_hyphen(i))
def count_equal(url):
    return url.count('=')
finaldata['count='] = finaldata['url'].apply(lambda i: count_equal(i))
def url_length(url):
    return len(str(url))
#Length of URL
finaldata['url_length'] = finaldata['url'].apply(lambda i: url_length(i))
#Hostname Length
def hostname_length(url):
    return len(urlparse(url).netloc)
finaldata['hostname_length'] = finaldata['url'].apply(lambda i: hostname_length(i))
finaldata.head()


,url,type,ipaddress,sus_url,dotnum,wwwnum,atnum,doubleslashnum,embeddomainnnum,abbreivation,count-https,count-http,count%,count?,count-,count=,url_length,hostname_length
0,br-icloud.com.br,unsafe,0,0,2,0,0,0,0,0,0,0,0,0,1,0,16,0
1,mp3raid.com/music/krizz_kaliko.html,safe,0,0,2,0,0,2,0,0,0,0,0,0,0,0,35,0
2,bopsecrets.org/rexroth/cr/1.htm,safe,0,0,2,0,0,3,0,0,0,0,0,0,0,0,31,0
3,http://www.garage-pirenne.be/index.php?option=...,unsafe,0,1,3,1,0,1,0,0,0,1,0,1,1,4,88,21
4,http://adventure-nicaragua.net/index.php?optio...,unsafe,0,1,2,0,0,1,0,0,0,1,0,1,1,3,235,23


In [ ]:
def suspicious_words(url):
    match = re.search('PayPal|login|signin|bank|account|update|free|lucky|service|bonus|ebayisapi|webscr',
                      url)
    if match:
        return 1
    else:
        return 0
finaldata['sus_url'] = finaldata['url'].apply(lambda i: suspicious_words(i))
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits
finaldata['count-digits']= finaldata['url'].apply(lambda i: digit_count(i))
def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters
finaldata['count-letters']= finaldata['url'].apply(lambda i: letter_count(i))
finaldata.head()

,url,type,ipaddress,sus_url,dotnum,wwwnum,atnum,doubleslashnum,embeddomainnnum,abbreivation,count-https,count-http,count%,count?,count-,count=,url_length,hostname_length,count-digits,count-letters
0,br-icloud.com.br,unsafe,0,0,2,0,0,0,0,0,0,0,0,0,1,0,16,0,0,13
1,mp3raid.com/music/krizz_kaliko.html,safe,0,0,2,0,0,2,0,0,0,0,0,0,0,0,35,0,1,29
2,bopsecrets.org/rexroth/cr/1.htm,safe,0,0,2,0,0,3,0,0,0,0,0,0,0,0,31,0,1,25
3,http://www.garage-pirenne.be/index.php?option=...,unsafe,0,0,3,1,0,1,0,0,0,1,0,1,1,4,88,21,7,63
4,http://adventure-nicaragua.net/index.php?optio...,unsafe,0,0,2,0,0,1,0,0,0,1,0,1,1,3,235,23,22,199


In [ ]:
!pip install tld
from urllib.parse import urlparse
from tld import get_tld
import os.path
#First Directory Length
def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
        # taking those before the first slash
    except:
        return 0
finaldata['fd_length'] = finaldata['url'].apply(lambda i: fd_length(i))
#Length of Top Level Domain (.com .edu)
finaldata['tld'] = finaldata['url'].apply(lambda i: get_tld(i,fail_silently=True))
def tld_length(tld):
    try:
        return len(tld)
    except:
        return -1
finaldata['tld_length'] = finaldata['tld'].apply(lambda i: tld_length(i))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 9.9 MB/s eta 0:00:00


In [ ]:
finaldata = finaldata.drop("tld",1)

<ipython-input-11-7abe8dd2e374>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  finaldata = finaldata.drop("tld",1)


In [ ]:
finaldata.columns

Index(['url', 'type', 'ipaddress', 'sus_url', 'dotnum', 'wwwnum', 'atnum',
       'doubleslashnum', 'embeddomainnnum', 'abbreivation', 'count-https',
       'count-http', 'count%', 'count?', 'count-', 'count=', 'url_length',
       'hostname_length', 'count-digits', 'count-letters', 'fd_length',
       'tld_length'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
finaldata["type_code"] = lb_make.fit_transform(finaldata["type"])
finaldata["type_code"].value_counts()

0    772924
1    298731
Name: type_code, dtype: int64

**5. แบ่งข้อมูลเตรียม train**

In [ ]:
X = finaldata[['ipaddress','sus_url', 'dotnum', 'wwwnum', 'atnum',
       'doubleslashnum', 'embeddomainnnum', 'abbreivation', 'count-https',
       'count-http', 'count%', 'count?', 'count-', 'count=', 'url_length',
       'hostname_length', 'sus_url', 'fd_length', 'tld_length', 'count-digits',
       'count-letters']]
#Target Variable
y = finaldata['type_code']

In [ ]:
X.columns

Index(['ipaddress', 'sus_url', 'dotnum', 'wwwnum', 'atnum', 'doubleslashnum',
       'embeddomainnnum', 'abbreivation', 'count-https', 'count-http',
       'count%', 'count?', 'count-', 'count=', 'url_length', 'hostname_length',
       'sus_url', 'fd_length', 'tld_length', 'count-digits', 'count-letters'],
      dtype='object')

แยก data train set, test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2,shuffle=True, random_state=5)

In [ ]:
w = finaldata[['url','ipaddress','sus_url', 'dotnum', 'wwwnum', 'atnum',
       'doubleslashnum', 'embeddomainnnum', 'abbreivation', 'count-https',
       'count-http', 'count%', 'count?', 'count-', 'count=', 'url_length',
       'hostname_length', 'sus_url', 'fd_length', 'tld_length', 'count-digits',
       'count-letters']]
#Target Variable
z = finaldata['type_code']

In [ ]:
w_train, w_test, z_train, z_test = train_test_split(w, z, stratify=z, test_size=0.2,shuffle=True, random_state=5)

In [ ]:
w_train.head()

,url,ipaddress,sus_url,dotnum,wwwnum,atnum,doubleslashnum,embeddomainnnum,abbreivation,count-https,...,count?,count-,count=,url_length,hostname_length,sus_url,fd_length,tld_length,count-digits,count-letters
43393,7wondersoftheworldpictures.com/,0,0,1,0,0,1,0,0,0,...,0,0,0,31,0,0,0,-1,1,28
74061,en.wikipedia.org/wiki/1967_NCAA_Men%27s_Divisi...,0,0,2,0,0,2,0,0,0,...,0,0,0,72,0,0,4,-1,6,55
127206,facebook.com/pages/Bloc-Qu%C3%A9b%C3%A9cois/10...,0,0,1,0,0,3,0,0,0,...,0,1,0,59,0,0,5,-1,19,31
526065,https://undermart.in/wp-content/themes/mapro/j...,0,0,2,0,0,5,0,0,1,...,0,1,0,66,12,0,10,2,0,55
540023,http://149.56.84.15/lmaoWTF/loligang.x86,1,0,4,0,0,2,0,0,0,...,0,0,0,40,12,0,7,-1,11,20


In [ ]:
w_train['url']

43393                       7wondersoftheworldpictures.com/
74061     en.wikipedia.org/wiki/1967_NCAA_Men%27s_Divisi...
127206    facebook.com/pages/Bloc-Qu%C3%A9b%C3%A9cois/10...
526065    https://undermart.in/wp-content/themes/mapro/j...
540023             http://149.56.84.15/lmaoWTF/loligang.x86
                                ...                        
469089    100percentfrenchmadeincanada.blogspot.com/2006...
83536     wiki.answers.com/Q/How_fast_can_the_bullet_tra...
18753                 aplictec.com.br/controller/emails/nD/
161441                                   wildcard.wf-tx.xyz
220116              docstoc.com/docs/6157795/Columbus__Ohio
Name: url, Length: 857324, dtype: object

**6.โหลด model**

โหลด pre-trained model AdamW

In [ ]:
!pip install transformers
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode the training data
train_encodings = tokenizer(w_train['url'].tolist(), truncation=True, padding=True, max_length=128)
train_labels = z_train.tolist()

# Tokenize and encode the test data
test_encodings = tokenizer(w_test['url'].tolist(), truncation=True, padding=True, max_length=128)
test_labels = z_test.tolist()

# Define the subset size
subset_size = 300000  # Adjust this value as desired

# Create smaller subsets of the training data
train_encodings = {key: value[:subset_size] for key, value in train_encodings.items()}
train_labels = train_labels[:subset_size]

# Create datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))

# Load the pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.0 MB/s eta 0:00:00


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**6. Train model ด้วยข้อมูล url 300000  url จาก train set ทั้งสิ้ง 10 epoch โดยใช้ batch = 70**

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Update loss function
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.fit(train_dataset.take(300000).shuffle(1000).batch(70), epochs=10, batch_size=70)

Epoch 1/10
4286/4286 [==============================] - 872s 192ms/step - loss: 0.0959 - accuracy: 0.9637
Epoch 2/10
4286/4286 [==============================] - 790s 184ms/step - loss: 0.0500 - accuracy: 0.9816
Epoch 3/10
4286/4286 [==============================] - 791s 184ms/step - loss: 0.0295 - accuracy: 0.9893
Epoch 4/10
4286/4286 [==============================] - 788s 184ms/step - loss: 0.0179 - accuracy: 0.9937
Epoch 5/10
4286/4286 [==============================] - 789s 184ms/step - loss: 0.0123 - accuracy: 0.9958
Epoch 6/10
4286/4286 [==============================] - 788s 184ms/step - loss: 0.0094 - accuracy: 0.9968
Epoch 7/10
4286/4286 [==============================] - 789s 184ms/step - loss: 0.0075 - accuracy: 0.9974
Epoch 8/10
4286/4286 [==============================] - 788s 184ms/step - loss: 0.0060 - accuracy: 0.9980
Epoch 9/10
4286/4286 [==============================] - 787s 184ms/step - loss: 0.0057 - accuracy: 0.9981
Epoch 10/10
4286/4286 [=======================

**7. Check accuracy จาก Test set**

In [ ]:
# Evaluate on the test dataset
sub_test_dataset = test_dataset.take(300000)
results = model.evaluate(sub_test_dataset.batch(70))
print("Test loss:", results[0])
print("Test accuracy:", results[1])



3062/3062 [==============================] - 215s 69ms/step - loss: 0.0993 - accuracy: 0.9807
Test loss: 0.09932880848646164
Test accuracy: 0.9807447195053101


**8.ลองใช้ model predict ลิ้งค์ต่างๆ**

In [ ]:
import re
from urllib.parse import urlparse
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

# Preprocessing function
def preprocess_link(link):
    # Convert the link to lowercase
    processed_link = link.lower()

    # Apply necessary transformations or feature engineering based on the existing code
    processed_link = re.sub(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', processed_link)
    processed_link = re.sub(r'^https?:\/\/', '', processed_link)
    processed_link = re.sub(r'www\.', '', processed_link)
    processed_link = re.sub(r'\.(com|org|net|mil|edu|COM|ORG|NET|MIL|EDU)$', '', processed_link)

    return processed_link

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load the trained model
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")


# Define the testing link
testing_link = "https://xn--trtkatkpay-3ubd.com/"

# Preprocess the testing link
processed_link = preprocess_link(testing_link)

# Tokenize and encode the testing link
inputs = tokenizer(processed_link, truncation=True, padding=True, max_length=128, return_tensors="tf")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Make predictions using the model
predictions = model.predict([input_ids, attention_mask])

# Assuming your model's output is a probability distribution, you can extract the predicted class or value
predicted_class = tf.argmax(predictions.logits, axis=-1).numpy()[0]

# Print the predicted class or value
print("Predicted class:", predicted_class)
if predicted_class == 0: print('the link is safe')
if predicted_class == 1: print('the link is unsafe')


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1/1 [==============================] - 3s 3s/step
Predicted class: 1
the link is unsafe


In [ ]:
import re
from urllib.parse import urlparse
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

# Preprocessing function
def preprocess_link(link):
    # Convert the link to lowercase
    processed_link = link.lower()

    # Apply necessary transformations or feature engineering based on the existing code
    processed_link = re.sub(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', processed_link)
    processed_link = re.sub(r'^https?:\/\/', '', processed_link)
    processed_link = re.sub(r'www\.', '', processed_link)
    processed_link = re.sub(r'\.(com|org|net|mil|edu|COM|ORG|NET|MIL|EDU)$', '', processed_link)

    return processed_link

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load the trained model
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")


# Define the testing link
testing_link = "https://www.youtube.com/"

# Preprocess the testing link
processed_link = preprocess_link(testing_link)

# Tokenize and encode the testing link
inputs = tokenizer(processed_link, truncation=True, padding=True, max_length=128, return_tensors="tf")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Make predictions using the model
predictions = model.predict([input_ids, attention_mask])

# Assuming your model's output is a probability distribution, you can extract the predicted class or value
predicted_class = tf.argmax(predictions.logits, axis=-1).numpy()[0]

# Print the predicted class or value
print("Predicted class:", predicted_class)
if predicted_class == 0: print('the link is safe')
if predicted_class == 1: print('the link is unsafe')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1/1 [==============================] - 3s 3s/step
Predicted class: 0
the link is safe


**9. save model ลง google drive**

In [ ]:
import tensorflow as tf
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to save the model in Google Drive
save_path = '/content/drive/MyDrive/bertmodel300000'

# Save the model
model.save(save_path)

# Print a message to indicate the successful saving of the model
print("Model saved successfully in Google Drive at:", save_path)


Mounted at /content/drive


Model saved successfully in Google Drive at: /content/drive/MyDrive/bertmodel300000
